In [1]:
## Create plot with sunburst diagram of classes in the dataset
%load_ext autoreload
%autoreload 2

In [1]:
import os, sys
sys.path.append('../scripts/')
import numpy as np
import matplotlib.pyplot as plt
import rasterio, rasterio.plot
import xarray as xr
import rioxarray as rxr
from tqdm import tqdm
import pandas as pd
import time, datetime, json
import geopandas as gpd
import loadpaths
import land_cover_analysis as lca
import land_cover_visualisation as lcv
import land_cover_models as lcm
import create_patches_from_polygons_data
from torch.utils.data import TensorDataset, DataLoader
import plotly.express as px


path_dict = loadpaths.loadpaths()

/home/tplas/anaconda3/envs/geo/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


adding scrub
adding wetland
adding transport


In [116]:
mapping_dict_to_full = {'C': 'Wood and Forest Land',
                            'D': 'Moor and Heath Land',
                            'E': 'Agro-Pastoral Land',
                            'F': 'Water and Wetland',
                            'G': 'Rock and Coastal Land',
                            'H': 'Developed Land',
                            'I': 'Unclassified Land',
                            '0': 'NO CLASS',
                            'U': 'NO CLASS'}

In [117]:
df_schema = lca.create_df_mapping_labels_2022_to_80s()
df_schema.drop(columns=['description_80s', 'code_80s', 'index_2022', 'index_80s'], inplace=True)
df_schema['main_class_code'] = df_schema['code_2022'].apply(lambda x: x[0])
df_schema['main_description'] = df_schema['main_class_code'].apply(lambda x: mapping_dict_to_full[x])

adding scrub
adding wetland
adding transport


In [133]:
classes_ignore = ['D4a', 'D4b', 'D8', 'F1', 'F3b', 'F3d', 'G2b', 'G3a', 'G3b', 'G3c', 'G3d']

In [132]:
df_schema.head(50)

,description_2022,code_2022,main_class_code,main_description
0,NO CLASS,0,0,NO CLASS
1,Broadleaved High Forest,C1,C,Wood and Forest Land
2,Coniferous High Forest,C2,C,Wood and Forest Land
3,Mixed High Forest,C3,C,Wood and Forest Land
4,Scrub,C4a,C,Wood and Forest Land
5,Scrub Pasture,C4b,C,Wood and Forest Land
6,Woodland/Scrub Edge,C4c,C,Wood and Forest Land
7,Clear Felled/New Plantings in Forest Areas,C5,C,Wood and Forest Land
8,Upland Heath,D1,D,Moor and Heath Land
9,Upland Grass Moor,D2b,D,Moor and Heath Land


In [176]:
data_plot = {'child': [], 'parent': [], 'values': []}
value_size = 2
for ii in range(len(df_schema)):
    code = df_schema['code_2022'].iloc[ii]
    if code == '0' or code in classes_ignore:
        continue
    if len(code) == 1:
        data_plot['child'].append(code)
        data_plot['parent'].append('LC schema')
        data_plot['values'].append(0) 
    elif len(code) == 2:
        data_plot['child'].append(code)
        # data_plot['parent'].append(mapping_dict_to_full[code[0]])
        data_plot['parent'].append(code[0])
        data_plot['values'].append(value_size)
    elif len(code) == 3:
        data_plot['child'].append(code)
        data_plot['parent'].append(code[:2])
        data_plot['values'].append(value_size)

        if code[:2] not in data_plot['child']:
            data_plot['child'].append(code[:2])
            data_plot['parent'].append(code[0])
            # data_plot['parent'].append(mapping_dict_to_full[code[0]])
            data_plot['values'].append(0)

list_low_level = list(set([x[0] for x in data_plot['child']]))
list_low_level.sort()
for low_level in list_low_level:
    if low_level not in data_plot['child']:
        data_plot['child'].append(low_level)
        # data_plot['child'].append(mapping_dict_to_full[low_level])
        data_plot['parent'].append('LC schema')
        data_plot['values'].append(0)

In [178]:
fig = px.sunburst(
    data_plot,
    names='child',
    parents='parent',
    values='values',
)
fig.update_traces(sort=False, selector=dict(type='sunburst'))
fig.show()